In [ ]:
import cv2
import io
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import re
from tqdm import tqdm
import warnings
import logging
warnings.filterwarnings('ignore')

In [ ]:
#Initialize log file to log all details about the song being processed
logging.basicConfig(filename="logs.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
def mp3_to_n_slice_np(file_path):
    """
    Function to convert .mp3 file to 10 slice mel spectrogram
    """
    y, sr = librosa.load(file_path)
    
    #Load mp3 file
    melspectrogram_array = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    
    #raise to db values
    mel = librosa.power_to_db(melspectrogram_array)

    #Covert mel values to image and slice into 10 parts
    plt.rcParams['figure.dpi'] = 100
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = float(mel.shape[1]) / float(100)
    fig_size[1] = float(mel.shape[0]) / float(100)
    plt.rcParams["figure.figsize"] = fig_size
    plt.axis('off')
    plt.axes([0., 0., 1., 1.0], frameon=False, xticks=[], yticks=[])
    librosa.display.specshow(mel, cmap='gray_r')
    buf = io.BytesIO()
    plt.savefig(buf, bbox_inches=None, pad_inches=0, format="jpg")
    plt.clf()
    plt.close()

    buf.seek(0)
    img_array = np.asarray(bytearray(buf.read()))
    img_col = cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)
    img_gray = cv2.cvtColor(img_col, cv2.COLOR_BGR2GRAY)
    buf.flush()
    buf.close()

    num_slices = int(img_gray.shape[1]/128)
    img_gray = img_gray[:,:(num_slices*128)]
    split_spectrum = np.hsplit(img_gray, num_slices)
    
    return split_spectrum, num_slices

# Load Metadata and Track Path

In [ ]:
# Get Genres and Track IDs from the tracks.csv file
filename_metadata = "./Dataset/fma_metadata/tracks.csv"
tracks = pd.read_csv(filename_metadata, header=2, low_memory=False)
tracks_array = tracks.values
tracks_id_array = tracks_array[:, 0]
tracks_genre_array = tracks_array[:, 40]
tracks_name_array = tracks_array[:, -1]

In [ ]:
#mp3 folder path
folder_sample = "./Dataset/fma_small"

# Get folders containing songs
directories = [d for d in os.listdir(folder_sample) 
              if os.path.isdir(os.path.join(folder_sample, d))]

# Processing of each song to feed into model

In [ ]:
X = []
y = []
name = []

#Iterate through each folder
for d in tqdm(directories):
    label_directory = os.path.join(folder_sample, d)
    
    #Get path of all .mp3 files for each folder
    file_names = [os.path.join(label_directory, f) for f in os.listdir(label_directory) if f.endswith(".mp3")]
    
    # Convert .mp3 files into mel-Spectograms
    for f in file_names:
        logger.info(f"Currently on {os.path.split(f)[1]}")
        try:
            split_output, n = mp3_to_n_slice_np(f)
            logger.info(f"The mp3 has been split into {n} slices")
            X.extend(split_output)

            track_id = int(os.path.split(f)[1].split(".")[0])
            track_index = list(tracks_id_array).index(track_id)
            output1 = [tracks_genre_array[track_index]]*n
            y.extend(output1)
            logger.info(f"Genre is {output1[0]}")
            
            output2 = [tracks_name_array[track_index]]*n
            name.extend(output2)
            logger.info(f"Song name is {output2[0]}")
        except Exception as e:
            logger.error(f"Could not load file due to {str(e)}")
print("Conversion Complete")

 28%|██▊       | 43/156 [2:04:41<5:18:19, 169.02s/it] --- Logging error ---
Traceback (most recent call last):
  File "C:\Users\jonat\Anaconda3\lib\logging\__init__.py", line 1037, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\jonat\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u011f' in position 47: character maps to <undefined>
Call stack:
  File "C:\Users\jonat\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jonat\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\jonat\Anaconda3\lib

  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\zmqshell.py", line 536, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2854, in run_cell
    raw_cell, store_history, silent, shell_futures)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2880, in _run_cell
    return runner(coro)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3057, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3248, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\Users\jonat\Anaconda3\lib\site-packages

  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 225, in wrapper
    runner = Runner(result, future, yielded)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 714, in __init__
    self.run()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 748, in run
    yielded = self.gen.send(value)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 365, in process_one
    yield gen.maybe_future(dispatch(*args))
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 209, in wrapper
    yielded = next(result)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 272, in dispatch_shell
    yield gen.maybe_future(handler(stream, idents, msg))
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 209, in wrapper
    yielded = next(result)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 542, in execute_req

  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\jonat\Anaconda3\lib\asyncio\base_events.py", line 539, in run_forever
    self._run_once()
  File "C:\Users\jonat\Anaconda3\lib\asyncio\base_events.py", line 1775, in _run_once
    handle._run()
  File "C:\Users\jonat\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\ioloop.py", line 690, in <lambda>
    lambda f: self._run_callback(functools.partial(callback, future))
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\ioloop.py", line 743, in _run_callback
    ret = callback()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 787, in inner
    self.run()
  File "C:\Users\jonat\

  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-67ac7afb6bca>", line 28, in <module>
    logger.info(f"Song name is {output2[0]}")
Message: 'Song name is Bańka mydlana'
Arguments: ()
 51%|█████▏    | 80/156 [4:03:26<2:23:30, 113.30s/it]--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\jonat\Anaconda3\lib\logging\__init__.py", line 1037, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\jonat\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u030c' in position 43: character maps to <undefined>
Call stack:
  File "C:\Users\jonat\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jonat\Anaconda3\lib\runpy.py", line 85, in _run_cod

...

  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\ioloop.py", line 743, in _run_callback
    ret = callback()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 787, in inner
    self.run()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 748, in run
    yielded = self.gen.send(value)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 378, in dispatch_queue
    yield self.process_one()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 225, in wrapper
    runner = Runner(result, future, yielded)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 714, in __init__
    self.run()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 748, in run
    yielded = self.gen.send(value)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 365, in process_one
    yield gen.maybe_future(dispatch(*args))
  File "C:\Users\jon

UnicodeEncodeError: 'charmap' codec can't encode characters in position 41-42: character maps to <undefined>
Call stack:
  File "C:\Users\jonat\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jonat\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\jonat\Anaconda3\lib\asyncio\base_events.py", line 539, in run_forever
    self._run_once()
  File "C:\Users\jonat\Anaconda3\lib\asyncio\base_even

  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2880, in _run_cell
    return runner(coro)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3057, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3248, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\Users\jonat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-67ac7afb6bca>", line 28, in <module>
    logger.info(f"Song name is {output2[0]}")
Message: 'Song name is Kerimoğlu Zeybek'
Arguments: ()
--- Logging error ---
Traceback (most recent call la

  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 365, in process_one
    yield gen.maybe_future(dispatch(*args))
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 209, in wrapper
    yielded = next(result)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 272, in dispatch_shell
    yield gen.maybe_future(handler(stream, idents, msg))
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 209, in wrapper
    yielded = next(result)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 542, in execute_request
    user_expressions, allow_stdin,
  File "C:\Users\jonat\Anaconda3\lib\site-packages\tornado\gen.py", line 209, in wrapper
    yielded = next(result)
  File "C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel\ipkernel.py", line 294, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "C:\Users\jonat\Anacon

Conversion Complete


# Split data into Train and Test

This data is not shuffled hence please shuffle it before feeding into model

In [ ]:
#Portion of samples to take as test for recommendation
start = 10_000
stop = 15_000

In [ ]:
X_test = np.array(X[start:stop])
y_test = np.array(y[start:stop])
name_test = np.array(name[start:stop])

X_train_par1 = np.array(X[:start])
X_train_par2 = np.array(X[stop:])
y_train_par1 = np.array(y[:start])
y_train_par2 = np.array(y[stop:])
name_train_par1 = np.array(name[:start])
name_train_par2 = np.array(name[stop:])

X_train = np.concatenate([X_train_par1, X_train_par2])
y_train = np.concatenate([y_train_par1, y_train_par2])
name_train = np.concatenate([name_train_par1, name_train_par2])

In [ ]:
if not os.path.exists("./train"):
    os.mkdir("./train")
np.save("./train/features.npy", X_train)
np.save("./train/classes.npy", y_train)
np.save("./train/names.npy", name_train)

In [ ]:
if not os.path.exists("./test"):
    os.mkdir("./test")
np.save("./test/features.npy", X_test)
np.save("./test/classes.npy", y_test)
np.save("./test/names.npy", name_test)